In [1]:
%load_ext autoreload
%autoreload 2
%cd ~/Research/Sriram/DeepSetRNN

/u/home/m/minhle/Research/Sriram/DeepSetRNN


In [2]:
import pandas as pd
from sklearn import metrics
import matplotlib.pyplot as plt
import numpy as np
from collections import defaultdict
import os
import string

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import logging
import os
import importlib

from src import convert_dot_format
from src import experiment_utils

In [3]:
logging.info("starting logger")
_LOGGER = logging.getLogger('preprocessing')
_LOGGER.setLevel(logging.DEBUG)

# Reading data

In [8]:
_ROOT_DATA_PATH = 'data/MIMIC3database/'
_PROCESSED_DATA_PATH = os.path.join(_ROOT_DATA_PATH, 'processed/MIMIC3EachPerson')

In [13]:
train_series_df = pd.read_csv(os.path.join(_PROCESSED_DATA_PATH, 'train_timeseries.csv'),
                              index_col=0,
                              nrows=1000)

train_labels_df = pd.read_csv(os.path.join(_PROCESSED_DATA_PATH, 'train_labels.csv'),
                              index_col=0,
                              nrows=1000)

In [15]:
train_labels_df

,Icustay,Ethnicity,Gender,Age,Height,Weight,Length of Stay,Mortality,Diagnosis 4019,Diagnosis 4280,...,Diagnosis 2869,Diagnosis 2763,Diagnosis 5770,Diagnosis V5865,Diagnosis 99662,Diagnosis 28860,Diagnosis 36201,Diagnosis 56210,SUBJECT_ID,ADMISSION_NUM
0,221100,0,1,71.990441,NaN,63.200001,2.1167,0,0,0,...,0,0,0,0,0,0,0,0,100,1
0,258326,0,2,69.754941,NaN,73.599998,36.7002,1,0,0,...,0,0,0,0,0,0,0,0,1000,1
0,216403,2,2,41.107157,NaN,NaN,1.4951,0,1,0,...,0,0,0,0,0,0,0,0,10003,1
0,268419,4,2,52.553274,NaN,NaN,0.7325,0,0,0,...,0,0,0,0,0,0,0,0,10004,1
0,279204,4,2,52.050853,NaN,100.000000,10.4665,0,1,0,...,0,0,0,0,0,0,0,0,10004,2
0,206504,2,1,70.684609,NaN,NaN,1.6325,0,0,1,...,0,0,0,0,1,0,0,0,10006,1
0,286008,4,1,55.332615,NaN,NaN,8.2914,0,0,0,...,0,0,0,0,0,0,0,0,10007,1
0,280464,4,1,53.756209,NaN,NaN,1.7126,0,0,0,...,0,0,0,0,0,0,0,0,10010,1
0,264446,0,1,87.145164,NaN,95.000000,2.6499,1,0,0,...,0,0,0,0,0,0,0,0,10013,1
0,240226,4,2,21.462955,NaN,NaN,0.9385,0,0,0,...,0,0,0,0,0,0,0,0,10014,1
